In [20]:
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import cv2
import os



In [21]:
severity_code = {'MILD': 0,
                'MODERATE': 1,
                'NORMAL': 2,
                'SEVERE': 3}

In [22]:
# specifying file locations

source_folder_train = '/Users/aravdhoot/Remote-PD-Detection/split_energy_images/Train/'
source_folder_val = '/Users/aravdhoot/Remote-PD-Detection/split_energy_images/Validation/'
channels = 3

In [23]:
# adding the image arrays to an array

resized_images_train = [
    cv2.resize(cv2.imread(os.path.join(source_folder_train, severity, video, image)), (256, 256))
    for severity in os.listdir(source_folder_train)
    for video in os.listdir(os.path.join(source_folder_train, severity))
    for image in os.listdir(os.path.join(source_folder_train, severity, video))
]

# formatting the array

num_samples = len(resized_images_train)
input_shape = (num_samples, 256, 256, channels)
input_array_train = np.empty(input_shape, dtype=np.uint8)

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 2 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 6 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 4 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 4 extraneous bytes before 

In [24]:
# adding the corresponding severity codes to the array

input_array_train = np.array(resized_images_train)
print(input_array_train.shape)
x_train = input_array_train

y_train = [
    severity_code[severity]
    for severity in os.listdir(source_folder_train)
    for video in os.listdir(os.path.join(source_folder_train, severity))
    for image in os.listdir(os.path.join(source_folder_train, severity, video))
]

(11808, 256, 256, 3)


In [25]:
# adding the image arrays to an array

resized_images_val = [
    cv2.resize(cv2.imread(os.path.join(source_folder_val, severity, video, image)), (256, 256))
    for severity in os.listdir(source_folder_val)
    for video in os.listdir(os.path.join(source_folder_val, severity))
    for image in os.listdir(os.path.join(source_folder_val, severity, video))
]

# formatting the array

num_samples = len(resized_images_val)
input_shape = (num_samples, 256, 256, channels)
input_array = np.empty(input_shape, dtype=np.uint8)

Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 3 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: 5 extraneous bytes before marker 0xd9
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corrupt JPEG data: premature end of data segment
Corr

In [26]:
# adding the corresponding severity codes to the array

input_array_val = np.array(resized_images_val)
print(input_array.shape)
x_val = input_array

y_val = [
    severity_code[severity]
    for severity in os.listdir(source_folder_val)
    for video in os.listdir(os.path.join(source_folder_val, severity))
    for image in os.listdir(os.path.join(source_folder_val, severity, video))
]

(1924, 256, 256, 3)


In [27]:
# randomly oversampling all the training images to have the same number of images

ros = RandomOverSampler(random_state=42)
x_train_2d = x_train.reshape(x_train.shape[0], -1)
X_resampled, y_resampled = ros.fit_resample(x_train_2d, y_train)
x_resampled_original = X_resampled.reshape(-1, x_train.shape[1], x_train.shape[2], x_train.shape[3])

In [28]:
# applying one-hot encoding on the class values

encoder = OneHotEncoder(sparse=False)
y_val = encoder.fit_transform(np.array(y_val).reshape(-1,1))
y_train = encoder.fit_transform(np.array(y_resampled).reshape(-1,1))

/Users/aravdhoot/opt/anaconda3/envs/deep-learning/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(
/Users/aravdhoot/opt/anaconda3/envs/deep-learning/lib/python3.10/site-packages/sklearn/preprocessing/_encoders.py:975: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [29]:
# normalizing the image arrays

x_resampled_original = x_resampled_original/255.0
x_val = x_val/255.0

In [30]:
from tensorflow.keras.layers.experimental.preprocessing import Rescaling
import tensorflow_addons as tfa
from vit_keras import vit

/Users/aravdhoot/opt/anaconda3/envs/deep-learning/lib/python3.10/site-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


In [37]:
model = Sequential([
    # Rescale pixel values
    Rescaling(1./255, input_shape=(256, 256, 3)),
    # Vision Transformer layer
    vit.vit_b16(
        image_size=256,
        activation='softmax',
        pretrained=True,
        include_top=False,
        pretrained_top=False,
        classes=4
    ),
    Flatten(),
    Dense(4, activation='softmax')
])

2024-02-06 18:58:40.227086: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-02-06 18:58:40.227298: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-02-06 18:58:40.227304: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-02-06 18:58:40.227533: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-02-06 18:58:40.227569: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)
/Users/aravdhoot/opt/anaconda3/envs/deep-learning/lib/python3.10/site-packages/vit_keras/utils.py:81: UserWarning: Resizing position embeddings from 24, 24 to 16, 16
  warnings.warn(

In [38]:
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Model summary to check architecture
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_6 (Rescaling)     (None, 256, 256, 3)       0         
                                                                 
 vit-b16 (Functional)        (None, 768)               85844736  
                                                                 
 flatten (Flatten)           (None, 768)               0         
                                                                 
 dense (Dense)               (None, 4)                 3076      
                                                                 
Total params: 85847812 (327.48 MB)
Trainable params: 85847812 (327.48 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [39]:
x_train = x_resampled_original

In [40]:
epochs1 = 36
epochs2= 4
batch_size = 32

x_val = x_val/255.0
chs=2000
history = {'loss': [], 'accuracy': [], 'val_loss': [], 'val_accuracy': []}
nchunk=x_train.shape[0]//chs
for i in range(0,nchunk+1):
    rl = i*chs
    rr = min((i+1)*chs, x_train.shape[0])
    print('doing chunk', rl,rr)
    x_traini = x_train[rl:rr,:,:]/255.0
    y_traini = y_train[rl:rr]
    historyi = model.fit(x_traini, y_traini, epochs=epochs1, batch_size=batch_size)
    historyi = model.fit(x_traini, y_traini, epochs=epochs2, batch_size=batch_size, validation_data=(x_val, y_val))
    history['accuracy'] = history['accuracy'] +historyi.history['accuracy'] 
    history['val_accuracy'] = history['val_accuracy'] +historyi.history['val_accuracy'] 
    history['loss'] = history['loss'] +historyi.history['loss']
    history['val_loss'] = history['val_loss'] +historyi.history['val_loss']

doing chunk 0 2000
